In [ ]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Lambda, InputLayer, concatenate
from keras.models import Model, Sequential
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

# Mathematical Underpinnings - Lab 7

## Task 2

### a)

In [ ]:
from scipy.stats import beta
from scipy import stats

In [ ]:
a = 2
b = 5

In [ ]:
def p_beta_d5(x, a, b):
    mix1 = [beta.pdf(x[:,i], a, b) for i in range(5)]
    mix2 = [beta.pdf(x[:,i], b, a) for i in range(5)]
    p_mix = np.prod(np.array(mix1), axis=0)/2 + np.prod(np.array(mix2), axis=0)/2
    return p_mix

In [ ]:
x_help = np.meshgrid(np.linspace(1e-5, 1-1e-5, 100), np.linspace(1e-5, 1-1e-5, 100)) 
x_grid = np.array([x_help[0].flatten(), x_help[1].flatten(), 1/2*np.ones(100*100), 1/2*np.ones(100*100), 1/2*np.ones(100*100)]).T

In [ ]:
p_grid = p_beta_d5(x_grid, a, b)

In [ ]:
plt.contourf(np.linspace(1e-10, 1-1e-10, 100), np.linspace(1e-10, 1-1e-10, 100), np.reshape(p_grid, (100, 100)))
plt.show()

### b)

In [ ]:
np.random.seed(123)
n = 10000
which_x_p = np.outer(np.random.binomial(1, 0.5, size=n), np.ones(5))
x_p = which_x_p*np.random.beta(a, b, size=(n, 5)) + (1 - which_x_p)*np.random.beta(b, a, size=(n, 5))
x_q = np.random.uniform(size=(n, 5))
x = np.concatenate((x_p, x_q), axis=0)
y = np.concatenate((1.0*np.ones(n), 0.0*np.zeros(n)))

**Nameless representation**

$$\sup_{\Phi > 0 } \left\{\mathbb{E}_p \log \Phi(X) - \mathbb{E}_q \Phi(X) +1\right\}$$

**Optimal function** 

$\textrm{arg max (nameless representation)} = \Phi^* = \frac{p}{q} = p$ (last equality due to the fact that $q$ is a density of the uniform distribution thus $q=1$)

In [ ]:
# plug in the optimal Phi = p in the formula for nameless representation and estimate expected values using averages
def no_name_representation(x, y):
    pass

In [ ]:
opt_val = no_name_representation(x, y)
opt_val

### c)

In [ ]:
def nameless_loss(y_true, y_pred):
    T_x = y_pred
    n1 = tf.math.reduce_sum(y_true)
    n2 = tf.math.reduce_sum(tf.subtract(tf.convert_to_tensor(1.0), y_true))
    first_term = tf.math.reduce_sum(tf.math.multiply(tf.math.log(T_x), y_true))/n1
    second_term = tf.math.reduce_sum(tf.math.multiply(T_x, tf.subtract(tf.convert_to_tensor(1.0), y_true)))/n2
    return -(first_term - second_term + 1) 

In [ ]:
model_inputs = keras.Input(shape=(5))
layer_out = Dense(16, activation='tanh')(model_inputs)
layer_out = Dense(16, activation='tanh')(layer_out)
layer_out = Dense(16, activation='tanh')(layer_out)
model_outputs = Dense(1, activation='softplus')(layer_out)

model = keras.Model(model_inputs, model_outputs)

optimizer = keras.optimizers.SGD(learning_rate=1e-2)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
model.compile(optimizer=optimizer, loss=nameless_loss)

In [ ]:
np.random.seed(123)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
np.random.seed(123)
hist = model.fit(x_train, y_train, epochs=2000, batch_size=256, validation_data=(x_test, y_test), callbacks=[es])

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.axhline(-opt_val, c="black")
plt.show()

### d)

In [ ]:
# Phi_pred = model's predictions - the model output optimal Phi, so it outputs the density p

In [ ]:
Phi_opt = p_beta_d5(x, a, b)

In [ ]:
plt.plot(Phi_opt[n:], Phi_pred[n:], "o")
plt.plot(Phi_opt[:n], Phi_pred[:n], "o")
plt.plot([0,30], [0, 30])
plt.show()


### e)

In [ ]:
kde = stats.gaussian_kde(x_p.T)

### f)

In [ ]:
p_est_grid = model.predict(x_grid)
p_est_grid = p_est_grid[:,0]

In [ ]:
p_kde_grid = kde(x_grid.T)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10,5))
axes[0].contourf(np.linspace(1e-10, 1-1e-10, 100), np.linspace(1e-10, 1-1e-10, 100), np.reshape(p_grid, (100, 100)))
axes[1].contourf(np.linspace(1e-10, 1-1e-10, 100), np.linspace(1e-10, 1-1e-10, 100), np.reshape(p_est_grid, (100, 100)))
axes[2].contourf(np.linspace(1e-10, 1-1e-10, 100), np.linspace(1e-10, 1-1e-10, 100), np.reshape(p_kde_grid, (100, 100)))
plt.show()

In [ ]:
np.random.seed(1234)
n = 1000
which_x_p_new_data = np.outer(np.random.binomial(1, 0.5, size=n), np.ones(5))
x_p_new_data = which_x_p_new_data*np.random.beta(a, b, size=(n, 5)) + (1 - which_x_p_new_data)*np.random.beta(b, a, size=(n, 5))

In [ ]:
# p_nd = 

In [ ]:
# p_est_nd = 

In [ ]:
# p_kde_nd = 

In [ ]:
ordering = np.argsort(p_nd)
plt.plot(p_est_nd[ordering], "o")
plt.plot(p_kde_nd[ordering], "o")
plt.plot(p_nd[ordering], "o")
plt.show()

In [ ]:
np.mean((p_kde_nd - p_nd)**2), np.mean((p_est_nd - p_nd)**2)